### Import Libraries, Models, and Environment Variables

In [1]:
# Import Libraries
import os
import pandas as pd
from dotenv import load_dotenv

# Import Langchain Libraries
from langchain_groq.chat_models import ChatGroq

# Load environment variables from .env file
load_dotenv()
Groq_Token = os.getenv("GROQ_API_KEY")

### Create the Dataset

In [12]:
# Define the data
dataset = pd.read_csv("examples.csv")
dataset["Color"] = dataset["Action"].apply(lambda action: action.split(": ")[1] if action != "UNKNOWN" else "UNKNOWN")

### Train the Model

In [13]:
model_name = "llama3-8b-8192"
chat_model = ChatGroq(model=model_name, api_key=Groq_Token, temperature=0)

In [14]:
def get_examples(dataset, color_name, num_examples=3):
    """
    Generate a few-shot example string for a specific color from a DataFrame.
    """
    color_data = dataset[dataset["Color"] == color_name]

    # Limit the number of examples to num_examples
    examples = []
    for index, line in color_data.head(num_examples).iterrows():
        example = f"""
        {index + 1}. User Prompt: {line['UserPrompt']}
        Available Colors: {line['AvailableColors']}
        Action: {line['Action']}
        """
        examples.append(example.strip())

    return "\n\n".join(examples)

def construct_prompt(dataset, user_prompt, available_colors):
    """
    Construct the prompt for color picking using the few-shot examples.
    """
    colors = dataset['Color'].unique()
    examples = ""

    for color in colors:
        examples += get_examples(dataset, color) + "\n\n"

    # Check if the requested color is available
    available_colors_set = set(available_colors)
    color_actions = [f"PICK UP COLOR: {color}" for color in available_colors_set]
    color_actions.append("UNKNOWN")

    prompt = f"""
    You are an expert in identifying colors based on user prompts.
    Based on the provided user prompts and few-shot examples, predict the action to pick up which color.

    Few-Shot Examples:
    {examples.strip()}

    User Prompt: {user_prompt}
    Available Colors: {', '.join(available_colors)}

    Predict the action as one of the following: {', '.join(color_actions)}.
    **Remember to output only the label in the specified format.**
    """
    return prompt.strip()

In [15]:
user_prompt = "The user wants to pick up the color that is closest to the color of the sky."
available_colors = ["black", "green", "red", "yellow"]
prompt = construct_prompt(dataset, user_prompt, available_colors)

try:
    response = chat_model.invoke(prompt)
    print(f"Response:\n{response.content}")
except Exception as e:
    print(f"Error: {e}")

Response:
PICK UP COLOR: yellow
